# Example EMA backtest using synthetic data

* Runs everything within a single notebook
    - Easy to read
    - Easy to test different functionalities of `tradeexecutor` library
* Uses synthetic random price dataa
    - No downloads needed
    - No API keys needed
* Long only
* No stop loss

## Set up

Set up strategy paramets that will decide its behavior

In [1]:
import datetime
import logging

import pandas as pd

from tradingstrategy.chain import ChainId
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.strategy_module import StrategyType, TradeRouting, ReserveCurrency

trading_strategy_cycle = CycleDuration.cycle_24h

# Strategy keeps its cash in BUSD
reserve_currency = ReserveCurrency.busd

# How much of the cash to put on a single trade
position_size = 0.10

#
# Strategy thinking specific parameter
#

slow_ema_candle_count = 20

fast_ema_candle_count = 5

# How many candles to extract from the dataset once
batch_size = 90

# Range of backtesting and synthetic data generation.
# Because we are using synthetic data actual dates do not really matter -
# only the duration

start_at = datetime.datetime(2021, 6, 1)
end_at = datetime.datetime(2022, 1, 1)


## Strategy function

This function decide what trades to take.

In [2]:

from tradeexecutor.state.visualisation import PlotKind
from tradeexecutor.state.trade import TradeExecution
from typing import List, Dict
from tradeexecutor.strategy.pricing_model import PricingModel
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradeexecutor.state.state import State
from tradingstrategy.universe import Universe


def decide_trades(
        timestamp: pd.Timestamp,
        universe: Universe,
        state: State,
        pricing_model: PricingModel,
        cycle_debug_data: Dict) -> List[TradeExecution]:
    """The brain function to decide the trades on each trading strategy cycle."""

    # The pair we are trading
    pair = universe.pairs.get_single()

    # How much cash we have in the hand
    cash = state.portfolio.get_current_cash()

    # Get OHLCV candles for our trading pair as Pandas Dataframe.
    # We could have candles for multiple trading pairs in a different strategy,
    # but this strategy only operates on single pair candle.
    # We also limit our sample size to N latest candles to speed up calculations.
    candles: pd.DataFrame = universe.candles.get_single_pair_data(timestamp)

    # We have data for open, high, close, etc.
    # We only operate using candle close values in this strategy.
    close = candles["close"]

    # Calculate exponential moving averages based on slow and fast sample numbers.
    # More information about calculating expotential averages in Pandas:
    #
    # - https://www.statology.org/exponential-moving-average-pandas/
    #
    # - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    #
    slow_ema_window = close.ewm(min_periods=slow_ema_candle_count).mean()
    fast_ema_window = close.ewm(min_periods=fast_ema_candle_count).mean()

    if len(slow_ema_window) == 0 or len(fast_ema_window) == 0:
        # We are backtesting early, so values not available yet
        return []

    slow_ema = slow_ema_window.iloc[-1]
    fast_ema = fast_ema_window.iloc[-1]
    # print("Slow EMA is", slow_ema)

    # Get the last close price from close time series
    # that's Pandas's Series object
    # https://pandas.pydata.org/docs/reference/api/pandas.Series.iat.html
    current_price = close.iloc[-1]

    # List of any trades we decide on this cycle.
    # Because the strategy is simple, there can be
    # only zero (do nothing) or 1 (open or close) trades
    # decides
    trades = []

    # Create a position manager helper class that allows us easily to create
    # opening/closing trades for different positions
    position_manager = PositionManager(timestamp, universe, state, pricing_model)

    if current_price >= slow_ema:
        # Entry condition:
        # Close price is higher than the slow EMA
        if not position_manager.is_any_open():
            buy_amount = cash * position_size
            trades += position_manager.open_1x_long(pair, buy_amount)
    elif fast_ema >= slow_ema:
        # Exit condition:
        # Fast EMA crosses slow EMA
        if position_manager.is_any_open():
            trades += position_manager.close_all()

    # Visualize strategy
    # See available Plotly colours here
    # https://community.plotly.com/t/plotly-colours-list/11730/3?u=miohtama
    visualisation = state.visualisation
    visualisation.plot_indicator(timestamp, "Slow EMA", PlotKind.technical_indicator_on_price, slow_ema, colour="darkblue")
    visualisation.plot_indicator(timestamp, "Fast EMA", PlotKind.technical_indicator_on_price, fast_ema, colour="mediumpurple")

    return trades

# Synthetic trading universe

We create a single Uniswap v2 like exchange with a single ETH-USDC trading pair.
This pair has generated noise like OHLCV trading data.

In [3]:

import random
from tradeexecutor.state.identifier import AssetIdentifier, TradingPairIdentifier
from tradingstrategy.candle import GroupedCandleUniverse
from tradeexecutor.testing.synthetic_ethereum_data import generate_random_ethereum_address
from tradeexecutor.testing.synthetic_exchange_data import generate_exchange
from tradeexecutor.testing.synthetic_price_data import generate_ohlcv_candles
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse, \
    create_pair_universe_from_code

def create_trading_universe() -> TradingStrategyUniverse:

    # Set up fake assets
    mock_chain_id = ChainId.ethereum
    mock_exchange = generate_exchange(
        exchange_id=random.randint(1, 1000),
        chain_id=mock_chain_id,
        address=generate_random_ethereum_address())
    usdc = AssetIdentifier(ChainId.ethereum.value, generate_random_ethereum_address(), "USDC", 6, 1)
    weth = AssetIdentifier(ChainId.ethereum.value, generate_random_ethereum_address(), "WETH", 18, 2)
    weth_usdc = TradingPairIdentifier(
        weth,
        usdc,
        generate_random_ethereum_address(),
        mock_exchange.address,
        internal_id=random.randint(1, 1000),
        internal_exchange_id=mock_exchange.exchange_id)

    time_bucket = TimeBucket.d1

    pair_universe = create_pair_universe_from_code(mock_chain_id, [weth_usdc])

    candles = generate_ohlcv_candles(time_bucket, start_at, end_at, pair_id=weth_usdc.internal_id)
    candle_universe = GroupedCandleUniverse.create_from_single_pair_dataframe(candles)

    universe = Universe(
        time_bucket=time_bucket,
        chains={mock_chain_id},
        exchanges={mock_exchange},
        pairs=pair_universe,
        candles=candle_universe,
        liquidity=None
    )

    return TradingStrategyUniverse(universe=universe, reserve_assets=[usdc])



## Run backtest

Run backtest using giving trading universe and strategy function.

In [4]:
from tradeexecutor.strategy.strategy_module import pregenerated_create_trading_universe
from tradeexecutor.testing.synthetic_exchange_data import generate_simple_routing_model
from tradeexecutor.backtest.backtest_runner import run_backtest_inline

universe = create_trading_universe()

start_candle, end_candle = universe.universe.candles.get_timestamp_range()
print(f"Our universe has synthetic candle data for the period {start_candle} - {end_candle}")

routing_model = generate_simple_routing_model(universe)

state, debug_dump = run_backtest_inline(
    start_at=start_at,
    end_at=end_at,
    client=None,  # None of downloads needed, because we are using synthetic data
    cycle_duration=CycleDuration.cycle_24h,  # Override to use 24h cycles despite what strategy file says
    decide_trades=decide_trades,
    universe=universe,
    initial_deposit=10_000,
    reserve_currency=ReserveCurrency.busd,
    trade_routing=TradeRouting.routing_model,
    routing_model=routing_model,
    log_level=logging.WARNING,
)


2022-06-27 01:53:52 tradeexecutor.backtest.backtest_execution          INFO     Initialising backtest execution model
2022-06-27 01:53:52 tradeexecutor.cli.loop                             TRADE    Preflight checks ok
2022-06-27 01:53:52 tradeexecutor.cli.loop                             INFO     Strategy is executed in backtesting mode, starting at 2021-06-01 00:00:00


Our universe has synthetic candle data for the period 2021-06-01 00:00:00 - 2021-12-31 00:00:00


  0%|          | 0/18489600 [00:00<?, ?it/s]

2022-06-27 01:53:52 tradeexecutor.cli.loop                             TRADE    Performing strategy tick #1 for timestamp 2021-06-01 00:00:00, unrounded time is 2021-06-01 00:00:00, live trading is False
2022-06-27 01:53:52 tradeexecutor.utils.timer                          INFO     Starting task strategy_tick at 2022-06-26 22:53:52.454954, context is {'clock': datetime.datetime(2021, 6, 1, 0, 0)}
2022-06-27 01:53:52 tradeexecutor.utils.timer                          INFO     Starting task sync_portfolio at 2022-06-26 22:53:52.455249, context is {}
2022-06-27 01:53:52 tradeexecutor.ethereum.wallet                      INFO     Portfolio reserve created. Asset: <USDC at 0xf73cd96cb3e16e017a223cb81c0d4c19609862a1>
2022-06-27 01:53:52 tradeexecutor.utils.timer                          INFO     Ended task sync_portfolio, took 0:00:00.000850
2022-06-27 01:53:52 tradeexecutor.utils.timer                          INFO     Starting task revalue_portfolio at 2022-06-26 22:53:52.456437, context 

ValueError: Must pass one of comass, span, halflife, or alpha

## Examine backtest results

Examine `state` that contains all actions the trade executor took.

In [ ]:
print(f"Positions taken: {len(list(state.portfolio.get_all_positions()))}")
print(f"Trades made: {len(list(state.portfolio.get_all_trades()))}")

# Visualise trading strategy

* Plot the OHLCV graph of the synthetic data
* Display trades on the graph

In [ ]:
from tradeexecutor.visual.single_pair import visualise_single_pair

figure = visualise_single_pair(state, universe.universe.candles)

figure.show()

## Finishing notes

In [ ]:
print("All ok")